In [1]:
import sys
import os
import numpy as np
import pandas as pd
import sqlalchemy as sa
sys.path.append(os.path.abspath(os.path.join('..','..','..')))
from pudl import pudl, ferc1, eia923, settings, constants
from pudl import models, models_ferc1, models_eia923
from pudl import clean_eia923, clean_ferc1, clean_pudl
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pudl_engine  = pudl.connect_db()

In [3]:
#Pull in the generation table
g_tbl = models.PUDLBase.metadata.tables['generation_eia923']
g_select = sa.sql.select([g_tbl,])
g = pd.read_sql(g_select, pudl_engine)
#Set the datetimeindex
g = g.set_index(pd.DatetimeIndex(g['report_date']))

In [4]:
#groupby plant_id and by year
g_yr = g.groupby([pd.TimeGrouper(freq='A'), 'plant_id', 'generator_id'])
#sum net_gen by year by plant
g_net_gen_gen = pd.DataFrame(g_yr.net_generation_mwh.sum())
g_net_gen_gen = g_net_gen_gen.reset_index(level=['generator_id'])
g_net_gen_gen

generator_id  net_generation_mwh
report_date plant_id                                 
2009-12-31  3                   1           221908.00
            3                   2           394031.00
            3                   3          1286393.00
            3                   4          1626547.00
            3                   5          4513101.00
            3                A1ST          1122697.00
            3                A2ST          1033733.00
            7                   1           212068.00
            7                   2            51262.00
            8                  10          3933248.00
            8                   6            98792.00
            8                   7            89753.00
            8                   8           566465.00
            8                   9           519196.00
            10                  1          1543167.00
            10                  2           821517.00
            26                  1           856407.00
            26                  2          1022033.00
            26                  3           657461.00
            26                  5          4706644.00
            26                ST4           790519.00
            47                  1           634534.00
            47                  2           424711.00
            47                  3           271306.00
            47                  4           454250.00
            47                  5          1154230.00
            50                  1            96134.00
            50                  2           111334.00
            50                  3            40763.00
            50                  4           162681.00
...                           ...                 ...
2016-12-31  58420            GEN2          1017786.00
            58426            GEN1          1466372.92
            58426            GEN2          1476164.04
            58503            GEN1           846452.00
            58503            GEN2           516330.00
            58557             GT1           527264.00
            58557             GT2           579861.00
            58557             ST1           872875.00
            58560            STG1           208730.00
            58697             CA1          1797795.00
            58697             CT1          1444793.00
            58697             CT2          1476393.00
            58944              G1            18752.00
            59254           GENS1            57455.04
            59254           GENT1            35410.82
            59254           GENT2            36945.36
            59254           GENT3            37096.68
            59254           GENT4            37694.20
            59338            CTG1           352966.00
            59338            CTG2           168324.00
            59338            CTG3           218810.00
            59338            CTG4           281232.00
            59338            ST10           339927.00
            59338             ST9           319721.00
            59784            CTG5           647756.00
            59784            CTG6           781011.00
            59784            ST11           958134.00
            60768            CTG3            61681.00
            60768            CTG4            10104.00
            60768            ST10            48106.00

[27006 rows x 2 columns]

In [5]:
#groupby plant_id and by year
g_net_gen_plant = g.groupby([pd.TimeGrouper(freq='A'), 'plant_id'])
#sum net_gen by year by plant and convert to datafram
g_net_gen_plant =  pd.DataFrame(g_net_gen_plant.net_generation_mwh.sum())
g_net_gen_plant

net_generation_mwh
report_date plant_id                    
2009-12-31  3               1.019841e+07
            7               2.633300e+05
            8               5.207454e+06
            10              2.364684e+06
            26              8.033064e+06
            47              2.939031e+06
            50              4.688202e+06
            51              4.465132e+06
            56              3.026254e+06
            59              5.916670e+05
            60              4.664970e+05
            79              2.093790e+05
            87              1.835862e+06
            108             2.655821e+06
            113             7.236124e+06
            116             7.872800e+04
            117             3.445720e+05
            118             6.610200e+04
            120             3.187580e+05
            126             6.822990e+05
            127             2.813612e+06
            130             1.461985e+07
            136             7.097411e+06
            141             6.012300e+04
            147             2.414980e+05
            160             2.041529e+06
            165             6.840435e+06
            167             2.010700e+04
            168             2.263000e+03
            169             4.452600e+04
...                                  ...
2016-12-31  57759           6.412573e+04
            57839           4.751779e+06
            57842           5.409940e+05
            57898           4.849110e+05
            57901           1.184687e+06
            57909           4.291820e+05
            57919           1.508510e+05
            57978           5.301870e+05
            58001           4.336063e+06
            58005           3.148465e+06
            58007           2.496130e+05
            58023           1.120440e+05
            58054           5.251280e+05
            58079           4.330434e+06
            58108           3.912340e+05
            58109           4.742922e+04
            58124           1.030750e+05
            58215           6.402250e+06
            58260           5.895472e+06
            58420           2.628001e+06
            58426           2.942537e+06
            58503           1.362782e+06
            58557           1.980000e+06
            58560           2.087300e+05
            58697           4.718981e+06
            58944           1.875200e+04
            59254           2.046021e+05
            59338           1.680980e+06
            59784           2.386901e+06
            60768           1.198910e+05

[11456 rows x 1 columns]

In [6]:
#Merge the summed net generation by generator with the summed net generation by plant
g_merged = g_net_gen_gen.merge(g_net_gen_plant, how="left", left_index=True, right_index=True)
g_merged['proportion_of_generation'] = (g_merged.net_generation_mwh_x/g_merged.net_generation_mwh_y)
#Remove the net generation columns
g_merged = g_merged.drop(['net_generation_mwh_x','net_generation_mwh_y'], axis=1)
g_merged

generator_id  proportion_of_generation
report_date plant_id                                       
2009-12-31  3                   1                  0.021759
            3                   2                  0.038637
            3                   3                  0.126137
            3                   4                  0.159490
            3                   5                  0.442530
            3                A1ST                  0.110085
            3                A2ST                  0.101362
            7                   1                  0.805332
            7                   2                  0.194668
            8                  10                  0.755311
            8                   6                  0.018971
            8                   7                  0.017235
            8                   8                  0.108780
            8                   9                  0.099702
            10                  1                  0.652589
            10                  2                  0.347411
            26                  1                  0.106610
            26                  2                  0.127228
            26                  3                  0.081844
            26                  5                  0.585909
            26                ST4                  0.098408
            47                  1                  0.215899
            47                  2                  0.144507
            47                  3                  0.092311
            47                  4                  0.154558
            47                  5                  0.392725
            50                  1                  0.020506
            50                  2                  0.023748
            50                  3                  0.008695
            50                  4                  0.034700
...                           ...                       ...
2016-12-31  58420            GEN2                  0.387285
            58426            GEN1                  0.498336
            58426            GEN2                  0.501664
            58503            GEN1                  0.621121
            58503            GEN2                  0.378879
            58557             GT1                  0.266295
            58557             GT2                  0.292859
            58557             ST1                  0.440846
            58560            STG1                  1.000000
            58697             CA1                  0.380971
            58697             CT1                  0.306166
            58697             CT2                  0.312863
            58944              G1                  1.000000
            59254           GENS1                  0.280814
            59254           GENT1                  0.173072
            59254           GENT2                  0.180572
            59254           GENT3                  0.181311
            59254           GENT4                  0.184232
            59338            CTG1                  0.209976
            59338            CTG2                  0.100134
            59338            CTG3                  0.130168
            59338            CTG4                  0.167302
            59338            ST10                  0.202220
            59338             ST9                  0.190199
            59784            CTG5                  0.271380
            59784            CTG6                  0.327207
            59784            ST11                  0.401413
            60768            CTG3                  0.514476
            60768            CTG4                  0.084277
            60768            ST10                  0.401248

[27006 rows x 2 columns]

In [7]:
#Pull in the fuel_receipts_cost table
frc_tbl = models.PUDLBase.metadata.tables['fuel_receipts_costs_eia923']
frc_select = sa.sql.select([frc_tbl,])
frc = pd.read_sql(frc_select, pudl_engine)
#Set the datetimeindex
frc = frc.set_index(pd.DatetimeIndex(frc['report_date']))
#groupby plant_id and by year
frc_yr_cost = frc.groupby([pd.TimeGrouper(freq='A'), 'plant_id'])
#sum fuel cost by year by plant
frc_yr_cost = frc_yr_cost.fuel_cost.sum()
#Convert back into a dataframe
frc_yr_cost_df = pd.DataFrame(frc_yr_cost)
frc_yr_cost_df

fuel_cost
report_date plant_id           
2009-12-31  3           28306.9
            7           63659.1
            8           53893.8
            9            4526.7
            10          16471.7
            26          49482.6
            47          41613.1
            50          60218.9
            51          10388.0
            54          15502.0
            56          39340.7
            59           1848.6
            60           4045.0
            64           6819.0
            75          34779.5
            87           8867.7
            96          28571.0
            99          85028.3
            108         12301.8
            113          8563.0
            116          3619.8
            117          5870.4
            118          4730.4
            120          4730.4
            124          6658.1
            126         16460.1
            127         21119.3
            130         57855.6
            136         29195.2
            141       7674428.8
...                         ...
2016-12-31  57068           NaN
            57070           NaN
            57267           NaN
            57349           NaN
            57482           NaN
            57515           NaN
            57555           NaN
            57664           NaN
            57839           NaN
            57842        3779.9
            57901           NaN
            57978        4090.0
            58001           NaN
            58005           NaN
            58079           NaN
            58215       11792.5
            58260        3106.4
            58266        4504.5
            58420           NaN
            58426           NaN
            58471           NaN
            58503        1252.2
            58557      320851.5
            58697       43433.2
            58835        3568.6
            59338           NaN
            59391        1242.0
            59784        4971.4
            59926        2456.8
            60768           NaN

[10017 rows x 1 columns]